In [9]:
import os
import pymysql
import pandas as pd

In [6]:
conn = pymysql.connect(host='localhost', user='localuser', db='country_club')

In [53]:
def run_query(query, save=False):
    df = pd.read_sql_query(query, conn)
    print(df.to_string(index=False))
    if save:
        return(df)

In [54]:
run_query('''

SELECT * 
  FROM Facilities

''')

facid             name  membercost  guestcost  initialoutlay  monthlymaintenance
    0   Tennis Court 1         5.0       25.0          10000                 200
    1   Tennis Court 2         5.0       25.0           8000                 200
    2  Badminton Court         0.0       15.5           4000                  50
    3     Table Tennis         0.0        5.0            320                  10
    4   Massage Room 1         9.9       80.0           4000                3000
    5   Massage Room 2         9.9       80.0           4000                3000
    6     Squash Court         3.5       17.5           5000                  80
    7    Snooker Table         0.0        5.0            450                  15
    8       Pool Table         0.0        5.0            400                  15


Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [55]:
run_query('''

SELECT name
  FROM Facilities
 WHERE membercost > 0

''')

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
  Squash Court


Q2: How many facilities do not charge a fee to members?

In [56]:
run_query('''

SELECT COUNT(name) AS "How many do not charge a fee to members?"
  FROM Facilities
 WHERE membercost > 0

''')

How many do not charge a fee to members?
                                       5


Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [57]:
run_query('''

SELECT facid, 
       name AS "facility name", 
       membercost AS "member cost",
       monthlymaintenance AS "monthly maintenance"
  FROM Facilities
 WHERE membercost > 0
   AND membercost < .2*monthlymaintenance

''')

facid   facility name  member cost  monthly maintenance
    0  Tennis Court 1          5.0                  200
    1  Tennis Court 2          5.0                  200
    4  Massage Room 1          9.9                 3000
    5  Massage Room 2          9.9                 3000
    6    Squash Court          3.5                   80


Q4: How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator.

In [58]:
run_query('''

SELECT *
  FROM Facilities
 WHERE facid in (1,5)

''')

facid            name  membercost  guestcost  initialoutlay  monthlymaintenance
    1  Tennis Court 2         5.0       25.0           8000                 200
    5  Massage Room 2         9.9       80.0           4000                3000


Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question.

In [59]:
run_query('''

SELECT name AS "facility name", 
       monthlymaintenance AS "monthly maintenance",
       CASE WHEN monthlymaintenance > 100 THEN 'expensive'
            WHEN monthlymaintenance <= 100 THEN 'cheap'
            ELSE 'error'
            END AS "cheap or expensive?"
  FROM Facilities

''')

facility name  monthly maintenance cheap or expensive?
 Tennis Court 1                  200           expensive
 Tennis Court 2                  200           expensive
Badminton Court                   50               cheap
   Table Tennis                   10               cheap
 Massage Room 1                 3000           expensive
 Massage Room 2                 3000           expensive
   Squash Court                   80               cheap
  Snooker Table                   15               cheap
     Pool Table                   15               cheap


In [61]:
df = run_query('''

SELECT *
  FROM Members
 LIMIT 4

''', save=True)

memid surname firstname                       address  zipcode       telephone recommendedby             joindate
    0   GUEST     GUEST                         GUEST        0  (000) 000-0000                2012-07-01 00:00:00
    1   Smith    Darren    8 Bloomsbury Close, Boston     4321    555-555-5555                2012-07-02 12:02:05
    2   Smith     Tracy  8 Bloomsbury Close, New York     4321    555-555-5555                2012-07-02 12:08:23
    3  Rownam       Tim        23 Highway Way, Boston    23423  (844) 693-0723                2012-07-03 09:32:15


In [62]:
df

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15


In [63]:
df = run_query('''

SELECT *
  FROM Bookings
 LIMIT 4

''', save=True)

bookid  facid  memid            starttime  slots
     0      3      1  2012-07-03 11:00:00      2
     1      4      1  2012-07-03 08:00:00      2
     2      6      0  2012-07-03 18:00:00      2
     3      7      1  2012-07-03 19:00:00      2


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.

In [71]:
run_query('''

  SELECT firstname AS "first name",
         surname AS "last name"
    FROM Members
   WHERE joindate = (SELECT MAX(joindate) FROM Members)

''')

first name last name
   Darren     Smith


Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [80]:
run_query('''

  SELECT DISTINCT CONCAT(m.firstname, ' ', m.surname) AS "member name", 
                  f.name AS "court name"
    FROM Members m
    JOIN Bookings b
    JOIN Facilities f
      ON m.memid = b.memid
     AND b.facid = f.facid
   WHERE SUBSTRING(f.name, 1, 12) = 'Tennis Court'
ORDER BY 1

''')

member name      court name
       Anne Baker  Tennis Court 2
       Anne Baker  Tennis Court 1
     Burton Tracy  Tennis Court 2
     Burton Tracy  Tennis Court 1
     Charles Owen  Tennis Court 2
     Charles Owen  Tennis Court 1
     Darren Smith  Tennis Court 2
    David Farrell  Tennis Court 1
    David Farrell  Tennis Court 2
      David Jones  Tennis Court 2
      David Jones  Tennis Court 1
     David Pinker  Tennis Court 1
    Douglas Jones  Tennis Court 1
    Erica Crumpet  Tennis Court 1
   Florence Bader  Tennis Court 1
   Florence Bader  Tennis Court 2
   Gerald Butters  Tennis Court 2
   Gerald Butters  Tennis Court 1
      GUEST GUEST  Tennis Court 1
      GUEST GUEST  Tennis Court 2
 Henrietta Rumney  Tennis Court 2
       Jack Smith  Tennis Court 2
       Jack Smith  Tennis Court 1
  Janice Joplette  Tennis Court 1
  Janice Joplette  Tennis Court 2
   Jemima Farrell  Tennis Court 1
   Jemima Farrell  Tennis Court 2
      Joan Coplin  Tennis Court 1
        John Hunt  T

Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [111]:
run_query('''

  SELECT f.name AS "facility name",
         CONCAT(m.firstname, ' ', m.surname) AS "member name",
         CASE WHEN m.memid = 0 THEN b.slots * f.guestcost
              ELSE b.slots * f.membercost
              END AS "total cost"
    FROM Members m
    JOIN Bookings b
    JOIN Facilities f
      ON m.memid = b.memid
     AND b.facid = f.facid
   WHERE CASE WHEN m.memid = 0 THEN b.slots * f.guestcost
              ELSE b.slots * f.membercost
              END > 30
     AND DATE(b.starttime) = '2012-09-14'
ORDER BY 3 DESC

''')

facility name     member name  total cost
Massage Room 2     GUEST GUEST       320.0
Massage Room 1     GUEST GUEST       160.0
Massage Room 1     GUEST GUEST       160.0
Massage Room 1     GUEST GUEST       160.0
Tennis Court 2     GUEST GUEST       150.0
Tennis Court 1     GUEST GUEST        75.0
Tennis Court 2     GUEST GUEST        75.0
Tennis Court 1     GUEST GUEST        75.0
  Squash Court     GUEST GUEST        70.0
Massage Room 1  Jemima Farrell        39.6
  Squash Court     GUEST GUEST        35.0
  Squash Court     GUEST GUEST        35.0


Q9: This time, produce the same result as in Q8, but using a subquery.

In [117]:
run_query('''

  SELECT t.fname AS "facility name", 
         t.mname AS "member name", 
         t.cost AS "total cost"
  FROM (
       SELECT f.name AS fname,
              CONCAT(m.firstname, ' ', m.surname) AS mname,
              CASE WHEN m.memid = 0 THEN b.slots * f.guestcost
                   ELSE b.slots * f.membercost
                   END AS cost,
              DATE(b.starttime) AS bdate
         FROM Members m
         JOIN Bookings b
         JOIN Facilities f
           ON m.memid = b.memid
          AND b.facid = f.facid
       ) t
   WHERE t.cost > 30
     AND t.bdate = '2012-09-14'
ORDER BY 3 DESC

''')

facility name     member name  total cost
Massage Room 2     GUEST GUEST       320.0
Massage Room 1     GUEST GUEST       160.0
Massage Room 1     GUEST GUEST       160.0
Massage Room 1     GUEST GUEST       160.0
Tennis Court 2     GUEST GUEST       150.0
Tennis Court 1     GUEST GUEST        75.0
Tennis Court 1     GUEST GUEST        75.0
Tennis Court 2     GUEST GUEST        75.0
  Squash Court     GUEST GUEST        70.0
Massage Room 1  Jemima Farrell        39.6
  Squash Court     GUEST GUEST        35.0
  Squash Court     GUEST GUEST        35.0


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [118]:
run_query('''

  SELECT f.name AS "facility name",
         SUM( CASE WHEN m.memid = 0 THEN b.slots * f.guestcost
              ELSE b.slots * f.membercost
              END ) AS "total revenue"
    FROM Members m
    JOIN Bookings b
    JOIN Facilities f
      ON m.memid = b.memid
     AND b.facid = f.facid
GROUP BY f.name
ORDER BY 2

''')

facility name  total revenue
   Table Tennis          180.0
  Snooker Table          240.0
     Pool Table          270.0
Badminton Court         1906.5
   Squash Court        13468.0
 Tennis Court 1        13860.0
 Tennis Court 2        14310.0
 Massage Room 2        14454.6
 Massage Room 1        50351.6
